In [174]:
ES_TRAIN_DATA_FILE = "./ES/train"
ES_TEST_DATA_FILE = "./ES/dev.in"
ES_ACTUAL_DATA_FILE = "./ES/dev.out"
ES_PART1_PREDICTED_DATA_FILE = "./ES/dev.p1.out"
ES_PART2_PREDICTED_DATA_FILE = "./ES/dev.p2.out"
ES_PART3_PREDICTED_DATA_FILE = "./ES/dev.p3.out"
ES_PART4_PREDICTED_DATA_FILE = "./ES/dev.p4.out"

RU_TRAIN_DATA_FILE = "./RU/train"
RU_TEST_DATA_FILE = "./RU/dev.in"
RU_ACTUAL_DATA_FILE = "./RU/dev.out"
RU_PART1_PREDICTED_DATA_FILE = "./RU/dev.p1.out"
RU_PART2_PREDICTED_DATA_FILE = "./RU/dev.p2.out"
RU_PART3_PREDICTED_DATA_FILE = "./RU/dev.p3.out"
RU_PART4_PREDICTED_DATA_FILE = "./RU/dev.p4.out"

UNK_WORD = "#UNK#"
START_TAG = "START"
STOP_TAG = "STOP"

# Part 1 - Emission Parameters, UNK Token

In [175]:
def _preprocess_training_file(file_path):
    with open(file_path, encoding="utf8") as f:
        data = f.read().splitlines()
        data[:] = [x for x in data if x]

    output = []
    for i in data:
        i = i.split(" ")
        if(len(i) > 2):
            i = [" ".join(i[0:len(i)-1]), i[len(i)-1]]
            output.append(i)
        else:
            output.append(i)
        
    return output

def _preprocess_testing_file(path):
    with open(path, encoding="utf8") as f:
        data = f.read().splitlines()

    output = []
    for word in data:
        # if word: # leave the newlines or not?
        output.append(word)

    return output

In [176]:
def get_emission_using_MLE(training, k = 1):
    tags = {}
    tags_to_word = {}
    emission = {}
    for data in training:
        word, tag = data[0], data[1]
        if tag in tags:
            tags[tag] += 1
        else:
            tags[tag] = 1
        
        tag_to_word = tuple((tag, word))
        if tag_to_word in tags_to_word:
            tags_to_word[tag_to_word] += 1
        else:
            tags_to_word[tag_to_word] = 1

    for key in tags_to_word.keys():
        emission[key] = tags_to_word[key] / (tags[key[0]] + k)
    for key in tags.keys():
        transition = tuple((key, UNK_WORD))
        emission[transition] = k / (tags[key] + k)
    # print(emission)
    return emission

In [177]:
def get_most_probable_tag(emission):
    highest_prob = {}
    output = {}
    for key, prob in emission.items():
        tag, word = key[0], key[1]
        if word not in highest_prob:
            highest_prob[word] = prob
            output[word] = tag
        else:
            if prob > highest_prob[word]:
                highest_prob[word] = prob
                output[word] = tag
    # print(output)
    return output

In [178]:
def get_unique_elements(lst):
    return list(set(list(itertools.chain.from_iterable(lst))))

def get_unique_tags(tags):
    tags_unique = get_unique_elements(tags)
    # print(f'tags_unique: {tags_unique}')
    tags_unique.sort()
    # print(f'tags_unique_sorted: {tags_unique}')
    tags_unique_with_start_stop = [START_TAG] + tags_unique + [STOP_TAG]
    # print(f'tags_unique_with_start_stop: {tags_unique_with_start_stop}\n')

    return tags_unique, tags_unique_with_start_stop


In [179]:
def write_to_predicted_file_part1(predicted_file, test_file, most_probable_tag):
    with open(predicted_file, "w", encoding="utf8") as f:
        for word in test_file:
            if len(word) > 0:
                try:
                    y = most_probable_tag[word]
                except:
                    y = most_probable_tag[UNK_WORD]
                f.write(f"{word} {y}\n")
            else: # leave the newlines??
                f.write("\n")

In [180]:
# Part 1 code for ES Dataset
ES_train_data = _preprocess_training_file(ES_TRAIN_DATA_FILE)
ES_test_data = _preprocess_testing_file(ES_TEST_DATA_FILE)

ES_emission_parameters = get_emission_using_MLE(ES_train_data)
ES_most_probable_tag = get_most_probable_tag(ES_emission_parameters)
write_to_predicted_file_part1(ES_PART1_PREDICTED_DATA_FILE, ES_test_data, ES_most_probable_tag)

# Part 1 code for RU Dataset
RU_train_data = _preprocess_training_file(RU_TRAIN_DATA_FILE)
RU_test_data = _preprocess_testing_file(RU_TEST_DATA_FILE)

RU_emission_parameters = get_emission_using_MLE(RU_train_data)
RU_most_probable_tag = get_most_probable_tag(RU_emission_parameters)
write_to_predicted_file_part1(RU_PART1_PREDICTED_DATA_FILE, RU_test_data, RU_most_probable_tag)

In [181]:
from EvalScript.evalResult import get_observed, get_predicted,compare_observed_to_predicted

def evaluateScores(actual_file, predicted_file):
    with open(predicted_file, encoding="utf8") as f:
        predicted = f.read().splitlines()

    with open(actual_file, encoding="utf8") as f:
        actual = f.read().splitlines()
   
    compare_observed_to_predicted(get_observed(actual), get_predicted(predicted))

evaluateScores(ES_ACTUAL_DATA_FILE, ES_PART1_PREDICTED_DATA_FILE)


#Entity in gold data: 255
#Entity in prediction: 1734

#Correct Entity : 205
Entity  precision: 0.1182
Entity  recall: 0.8039
Entity  F: 0.2061

#Correct Sentiment : 112
Sentiment  precision: 0.0646
Sentiment  recall: 0.4392
Sentiment  F: 0.1126


# Part 2i - Transition Parameters

In [182]:
import itertools
def _preprocess_training_file(training_file):
    tags = []
    tags_with_start_stop = []
    words = []

    with open(training_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            tags_list = []
            tags_with_start_stop_list = []
            words_list = []

            for word_tag in line.split("\n"):
                i = word_tag.split(" ")

                if len(i) > 2:
                    i = [" ".join(i[0:len(i)-1]), i[len(i)-1]]

                word, tag = i[0], i[1]
                words_list.append(word)
                tags_list.append(tag)

            tags.append(tags_list)
            tags_with_start_stop_list = [START_TAG] + tags_list + [STOP_TAG]
            tags_with_start_stop.append(tags_with_start_stop_list)
            words.append(words_list)
    
    # print(f'tags: {tags[0:3]}')
    # print(f'tags_with_start_stop: {tags_with_start_stop[0:3]}')
    # print(f'words: {words[0:3]}\n')
    return tags, tags_with_start_stop, words

def _preprocess_test_file(testing_file):
    test_words = []

    with open(testing_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            word_list = []
            for word in line.split("\n"):
                word_list.append(word)
            test_words.append(word_list)

    return test_words

In [183]:
def get_transition_pairs(tags):
    transition_pair_count = {}

    for tag in tags:
        for tag1, tag2 in zip(tag[:-1], tag[1:]):
            transition_pair = (tag1, tag2)
            if transition_pair in transition_pair_count:
                transition_pair_count[transition_pair] += 1
            else:
                transition_pair_count[transition_pair] = 1

    return transition_pair_count

In [184]:
def get_transition_triplets(tags):
    transition_triplet_count = {}

    for tag in tags:
        for tag1, tag2, tag3 in zip(tag[:-2], tag[1:-1], tag[2:]):
            transition_triplet = (tag1, tag2, tag3)
            if transition_triplet in transition_triplet_count:
                transition_triplet_count[transition_triplet] += 1
            else:
                transition_triplet_count[transition_triplet] = 1

    return transition_triplet_count

In [185]:
def count_y(tag, tags):
    tags_flattened = list(itertools.chain.from_iterable(tags))
    return tags_flattened.count(tag)

In [186]:
def get_transition_pairs_using_MLE(tags_unique_with_start_stop, transition_pair_count,
                          tags_with_start_stop):

    transition = {}
    #create matrix dimensions
    for tag1 in tags_unique_with_start_stop[:-1]:
        transition_row = {}
        for tag2 in tags_unique_with_start_stop[1:]:
            transition_row[tag2] = 0.0
        transition[tag1] = transition_row

    # populate transition parameters with counts
    for tag1, tag2 in transition_pair_count:
        transition[tag1][tag2] += transition_pair_count[(tag1, tag2)]

    # divide transition_count by count_yi, to get probability
    for tag1, transition_row in transition.items():
        count_yi = count_y(tag1, tags_with_start_stop)

        # words in training set
        for tag2, transition_count in transition_row.items():
            if count_yi == 0:
                transition[tag1][tag2] = 0.0
            else:
                transition[tag1][tag2] = transition_count / count_yi

    return transition

In [187]:
def get_transition_triplets_using_MLE(tags_unique_with_start_stop, transition_triplet_count, tags_with_start_stop):
    transition = {}

    # Tags (u -> v -> w)
    for u in tags_unique_with_start_stop[:-1]:  # omit STOP for first tag
        transition_first_tag = {}
        for v in tags_unique_with_start_stop[1:-1]:  # omit START and STOP for second tag
            transition_second_tag = {}
            for w in tags_unique_with_start_stop[1:]:  # omit START for third tag
                transition_second_tag[w] = 0.0
            transition_first_tag[v] = transition_second_tag
        transition[u] = transition_first_tag

    # fill up transition parameters
    for u, v, w in transition_triplet_count:
        transition[u][v][w] += transition_triplet_count[(u, v, w)]

    # divide transition_count by count_yi, to get probability
    for u, transition_first_tag in transition.items():
        for v, transition_second_tag in transition_first_tag.items():
            count_yi = count_y(u, tags_with_start_stop)

            for w, instance_in_training_set in transition_second_tag.items():
                if count_yi == 0:
                    transition[u][v][w] = 0.0
                else:
                    transition[u][v][w] = instance_in_training_set / count_yi

    return transition


In [188]:
# Get ES tags
ES_tags, ES_tags_with_start_stop, ES_train_words = _preprocess_training_file(ES_TRAIN_DATA_FILE)
ES_tags_unique, ES_tags_unique_with_start_stop = get_unique_tags(ES_tags)

# Get ES words
ES_test_words = _preprocess_test_file(ES_TEST_DATA_FILE)
ES_unique_words = get_unique_elements(ES_train_data)

# Get ES transition pairs and triplets
ES_transition_pair_count = get_transition_pairs(ES_tags_with_start_stop)
ES_transition_pair_parameters = get_transition_pairs_using_MLE(ES_tags_unique_with_start_stop, ES_transition_pair_count, ES_tags_with_start_stop)
ES_transition_triplet_count = get_transition_triplets(ES_tags_with_start_stop)
ES_transition_triplet_parameters = get_transition_triplets_using_MLE(ES_tags_unique_with_start_stop, ES_transition_triplet_count, ES_tags_with_start_stop)

print(ES_transition_pair_parameters)
print(ES_transition_triplet_parameters)

# Get RU tags
RU_tags, RU_tags_with_start_stop, RU_train_words = _preprocess_training_file(RU_TRAIN_DATA_FILE)
RU_tags_unique, RU_tags_unique_with_start_stop = get_unique_tags(RU_tags)

# Get RU words
RU_test_words = _preprocess_test_file(RU_TEST_DATA_FILE)
RU_unique_words = get_unique_elements(RU_train_data)

# Get RU transition pairs and triplets
RU_transition_pair_count = get_transition_pairs(RU_tags_with_start_stop)
RU_transition_pair_parameters = get_transition_pairs_using_MLE(RU_tags_unique_with_start_stop, RU_transition_pair_count, RU_tags_with_start_stop)
RU_transition_triplet_count = get_transition_triplets(RU_tags_with_start_stop)
RU_transition_triplet_parameters = get_transition_triplets_using_MLE(RU_tags_unique_with_start_stop, RU_transition_triplet_count, RU_tags_with_start_stop)

{'START': {'B-negative': 0.013075060532687652, 'B-neutral': 0.004842615012106538, 'B-positive': 0.053268765133171914, 'I-negative': 0.0, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.9288135593220339, 'STOP': 0.0}, 'B-negative': {'B-negative': 0.0, 'B-neutral': 0.0, 'B-positive': 0.0, 'I-negative': 0.18181818181818182, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.8088578088578089, 'STOP': 0.009324009324009324}, 'B-neutral': {'B-negative': 0.0, 'B-neutral': 0.0, 'B-positive': 0.0, 'I-negative': 0.0, 'I-neutral': 0.18823529411764706, 'I-positive': 0.0, 'O': 0.8117647058823529, 'STOP': 0.0}, 'B-positive': {'B-negative': 0.0, 'B-neutral': 0.0007849293563579278, 'B-positive': 0.0015698587127158557, 'I-negative': 0.0, 'I-neutral': 0.0, 'I-positive': 0.1271585557299843, 'O': 0.8634222919937206, 'STOP': 0.00706436420722135}, 'I-negative': {'B-negative': 0.0, 'B-neutral': 0.0, 'B-positive': 0.0, 'I-negative': 0.6593886462882096, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.3406113537117904, 'STOP':

# Part 4 - Trigram HMM + Viterbi Algorithm

In [189]:
import math
import sys

word_output_list = []  # list of tuple(word, predicted_tag) for writing to file
viterbi_values = {}  # key: (n, tag)  value: float 

def generate_viterbi_values(n, current_tag, word_list, words_unique, tags_unique, emission_params, transmission_pair_params, transmission_triplet_params):
    global viterbi_values
    current_max_viterbi_value = -sys.float_info.max  # Smallest negative float

    if n == 0:
        return

    # Use emission pair for n == 1
    if n == 1:
        try:
            if word_list[n-1] in words_unique:
                try:
                    current_max_viterbi_value = math.log(emission_params[(current_tag, word_list[n-1])] * transmission_pair_params[START_TAG][current_tag])
                except KeyError:
                    current_max_viterbi_value = -sys.float_info.max
            else:
                current_max_viterbi_value = math.log(emission_params[(current_tag, UNK_WORD)] * transmission_pair_params[START_TAG][current_tag])
        except ValueError:
            current_max_viterbi_value = -sys.float_info.max
        
        viterbi_values[(n, current_tag)] = current_max_viterbi_value
        return

    # Recursive call to generate viterbi_values for (n-1, tag)
    for tag in tags_unique:
        if (n-1, tag) not in viterbi_values:
            generate_viterbi_values(n-1, tag, word_list, words_unique, tags_unique, emission_params, transmission_pair_params, transmission_triplet_params)
            
    # Handle n == 2 where first tag in triplet is START
    if n == 2:
        for tag in tags_unique:
            try:
                if word_list[n-1] in words_unique:
                    try:
                        value = viterbi_values[(n-1, tag)] + math.log(emission_params[(current_tag, word_list[n-1])] * transmission_triplet_params[START_TAG][tag][current_tag])
                    except KeyError:
                        continue
                else:
                    value = viterbi_values[(n-1, tag)] + math.log(emission_params[(current_tag, UNK_WORD)] * transmission_triplet_params[START_TAG][tag][current_tag])
            except ValueError:
                continue
            current_max_viterbi_value = max(current_max_viterbi_value, value)
        
        viterbi_values[(n, current_tag)] = current_max_viterbi_value
        return

    # Use viterbi values from n-1 to generate current viterbi value
    for tag1 in tags_unique:
        for tag2 in tags_unique:
            try:
                if word_list[n-1] in words_unique:
                    try:
                        value = viterbi_values[(n-1, tag2)] + math.log(emission_params[(current_tag, word_list[n-1])] * transmission_triplet_params[tag1][tag2][current_tag])
                    except KeyError:
                        continue
                else:
                    value = viterbi_values[(n-1, tag2)] + math.log(emission_params[(current_tag, UNK_WORD)] * transmission_triplet_params[tag1][tag2][current_tag])
            except ValueError:
                continue
            
            current_max_viterbi_value = max(current_max_viterbi_value, value)
    
    viterbi_values[(n, current_tag)] = current_max_viterbi_value


# function to kickstart viterbi recursive chain, and add (n+1, STOP) to veterbi_values
def start_viterbi(word_list, words_unique, tags_unique, emission_params, transmission_pair_params, transmission_triplet_params):
    global viterbi_values
    max_final_viterbi_value = -sys.float_info.max

    n = len(word_list)

    # Recursive call to generate viterbi_values for (n, tag)
    for tag in tags_unique:
        generate_viterbi_values(n, tag, word_list, words_unique, tags_unique, emission_params, transmission_pair_params, transmission_triplet_params)

    # Use viterbi values from n to generate viterbi value for (n+1, STOP)
    for tag1 in tags_unique:
        for tag2 in tags_unique:
            try:
                value = viterbi_values[(n, tag2)] + math.log(transmission_triplet_params[tag1][tag2][STOP_TAG])
            except ValueError:
                continue
            max_final_viterbi_value = max(max_final_viterbi_value, value)

    viterbi_values[(n+1, STOP_TAG)] = max_final_viterbi_value


def generate_predictions_viterbi(word_list, words_unique, tags_unique, emission_params, transmission_pair_params, transmission_triplet_params):
    global viterbi_values

    n = len(word_list)

    generated_tag_list = ['' for i in range(n)]

    current_best_tag = 'O'
    current_best_tag_value = -sys.float_info.max

    # Handle case where word is length 1
    if n == 1:
        for tag in tags_unique:
            try:
                value = viterbi_values[(1, tag)] + math.log(transmission_triplet_params[START_TAG][tag][STOP_TAG])
            except ValueError:
                continue
            if value > current_best_tag_value:
                current_best_tag = tag
                current_best_tag_value = value

        generated_tag_list[0] = current_best_tag
        return generated_tag_list


    # Compute tag for n
    for tag1 in tags_unique:
        for tag2 in tags_unique:
            try:
                value = viterbi_values[(n, tag2)] + math.log(transmission_triplet_params[tag1][tag2][STOP_TAG])
            except ValueError:
                continue
            if value > current_best_tag_value:
                current_best_tag = tag2
                current_best_tag_value = value

    generated_tag_list[n-1] = current_best_tag

    # Generate predictions starting from n-1 going down to 2
    for i in range(n-1, 1, -1):
        current_best_tag = 'O'
        current_best_tag_value = -sys.float_info.max

        for tag1 in tags_unique:
            for tag2 in tags_unique:
                try:
                    value = viterbi_values[(i, tag2)] + math.log(transmission_triplet_params[tag1][tag2][generated_tag_list[i]])
                except ValueError:
                    continue
                if value > current_best_tag_value:
                    current_best_tag = tag2
                    current_best_tag_value = value

        generated_tag_list[i-1] = current_best_tag

    # Lastly, generate prediction for n = 1
    current_best_tag = 'O'
    current_best_tag_value = -sys.float_info.max

    for tag in tags_unique:
        try:
            value = viterbi_values[(1, tag)] + math.log(transmission_triplet_params[START_TAG][tag][generated_tag_list[1]])
        except ValueError:
            continue
        if value > current_best_tag_value:
            current_best_tag = tag
            current_best_tag_value = value

    generated_tag_list[0] = current_best_tag

    return generated_tag_list


def write_to_predicted_file_part4(predicted_file, words_list, tags_list):
    assert len(words_list) == len(tags_list)

    with open(predicted_file, "w", encoding="utf8") as f:
        for words, tags in zip(words_list, tags_list):  # Unpack all sentences and list of tags
            assert len(words) == len(tags)
            for word, tag in zip (words, tags):  # Unpack all words and tags
                f.write(f"{word} {tag}\n")
            f.write("\n")


In [190]:
# Run and output Viterbi for ES
ES_predicted_tags_list = []
for word in ES_test_words:
    viterbi_values = {}
    start_viterbi(word, ES_unique_words, ES_tags_unique, ES_emission_parameters, ES_transition_pair_parameters, ES_transition_triplet_parameters)
    ES_generated_tag_list = generate_predictions_viterbi(word, ES_unique_words, ES_tags_unique, ES_emission_parameters, ES_transition_pair_parameters, ES_transition_triplet_parameters)
    ES_predicted_tags_list.append(ES_generated_tag_list)

write_to_predicted_file_part4(ES_PART4_PREDICTED_DATA_FILE, ES_test_words, ES_predicted_tags_list)


# Run and output Viterbi for RU
RU_predicted_tags_list = []
for word in RU_test_words:
    viterbi_values = {}
    start_viterbi(word, RU_unique_words, RU_tags_unique, RU_emission_parameters, RU_transition_pair_parameters, RU_transition_triplet_parameters)
    RU_generated_tag_list = generate_predictions_viterbi(word, RU_unique_words, RU_tags_unique, RU_emission_parameters, RU_transition_pair_parameters, RU_transition_triplet_parameters)
    RU_predicted_tags_list.append(RU_generated_tag_list)

write_to_predicted_file_part4(RU_PART4_PREDICTED_DATA_FILE, RU_test_words, RU_predicted_tags_list)


In [192]:
from EvalScript.evalResult import get_observed, get_predicted,compare_observed_to_predicted

def evaluateScores(actual_file, predicted_file):
    with open(predicted_file, encoding="utf8") as f:
        predicted = f.read().splitlines()

    with open(actual_file, encoding="utf8") as f:
        actual = f.read().splitlines()
   
    compare_observed_to_predicted(get_observed(actual), get_predicted(predicted))

evaluateScores(ES_ACTUAL_DATA_FILE, ES_PART4_PREDICTED_DATA_FILE)
# evaluateScores(RU_ACTUAL_DATA_FILE, RU_PART4_PREDICTED_DATA_FILE)


#Entity in gold data: 255
#Entity in prediction: 73

#Correct Entity : 28
Entity  precision: 0.3836
Entity  recall: 0.1098
Entity  F: 0.1707

#Correct Sentiment : 24
Sentiment  precision: 0.3288
Sentiment  recall: 0.0941
Sentiment  F: 0.1463

#Entity in gold data: 461
#Entity in prediction: 237

#Correct Entity : 87
Entity  precision: 0.3671
Entity  recall: 0.1887
Entity  F: 0.2493

#Correct Sentiment : 61
Sentiment  precision: 0.2574
Sentiment  recall: 0.1323
Sentiment  F: 0.1748
